In [8]:
results_file = '11_16-12_22isoerror_results.np'

In [9]:
!sshpass -p 'password' scp rns365@prince.hpc.nyu.edu:/home/rns365/quality_quantity_tradeoff/{results_file} /Users/rubenstern/Desktop/

In [10]:
import pickle
import numpy as np
import torch

import plotly
plotly.tools.set_credentials_file(username='rubenstern', api_key='tsHqUIBaeDLvOjJAiWMI')

import plotly.plotly as py
import plotly.graph_objs as go

In [31]:
with open(results_file, 'rb') as fd:
    data = pickle.load(fd)
    
data = np.array([[100*(1-result['quality']), result['quantity'], float(result['accuracy'])] for result in data])

In [32]:
qualities = np.sort(np.unique(data[:,0]))
quantities = np.sort(np.unique(data[:,1]))

In [33]:
qualities

array([  60.,   65.,   70.,   75.,   80.,   85.,   90.,   95.,  100.])

In [34]:
quantities

array([  2000.,   3000.,   4000.,   5000.,   6000.,   7000.,   8000.,
         9000.,  10000.])

In [35]:
data

array([[  6.00000000e+01,   2.00000000e+03,   5.00000000e-01],
       [  6.50000000e+01,   2.00000000e+03,   5.00000000e-01],
       [  7.00000000e+01,   2.00000000e+03,   5.33999979e-01],
       [  7.50000000e+01,   2.00000000e+03,   5.49000025e-01],
       [  8.00000000e+01,   2.00000000e+03,   5.73499978e-01],
       [  8.50000000e+01,   2.00000000e+03,   5.95499992e-01],
       [  9.00000000e+01,   2.00000000e+03,   6.21999979e-01],
       [  9.50000000e+01,   2.00000000e+03,   6.24499977e-01],
       [  1.00000000e+02,   2.00000000e+03,   6.49500012e-01],
       [  6.00000000e+01,   3.00000000e+03,   5.00000000e-01],
       [  6.50000000e+01,   3.00000000e+03,   5.00000000e-01],
       [  7.00000000e+01,   3.00000000e+03,   5.51999986e-01],
       [  7.50000000e+01,   3.00000000e+03,   5.93500018e-01],
       [  8.00000000e+01,   3.00000000e+03,   5.77499986e-01],
       [  8.50000000e+01,   3.00000000e+03,   6.08500004e-01],
       [  9.00000000e+01,   3.00000000e+03,   6.2900000

In [36]:
data[58,2] = (data[57,2]+data[59,2])/2

In [37]:
def generate_isoerror_plot(data):
    acc_array = []
    for q in qualities:
        selected_quality = data[data[:,0]==q]
        line_acc = selected_quality[selected_quality[:,1].argsort()][:,2]
        acc_array.append(list(line_acc))
    iso_data=[
        go.Contour(
            z=acc_array,
            x=quantities,
            y=qualities,
            contours=dict(
                coloring ='heatmap',
                showlabels = True,
                labelfont = dict(
                    family = 'Raleway',
                    size = 12,
                    color = 'white',
                )
            )
        )
    ]
    return iso_data


In [39]:
iso_data = generate_isoerror_plot(data)

In [40]:
py.iplot(iso_data)